# Preanálisis

In [1]:
# Librerías de siempre
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import os
import datetime
from time import time
import statsmodels.api as sm
import itertools
from random import shuffle
from statsmodels.formula.api import ols
from scipy import stats

In [21]:
def leer_efectos(datadir,
                 ligas):
    dict_replaces = {'_mean': '',
                     '_':' ',
                     'dificil':'Difícil',
                     'dificl' : 'Difícil',
                     'facil': 'Fácil',
                     'regular' : 'Regular',
                     'prom': 'Promedio',
                     'posicion': 'Posición',
                     'primer' : 'Primer',
                     'ronda' : 'Ronda',
                     'gral': 'Gral.',
                     'categoria': 'Categoría',
                     'local': 'Local',
                     'visita': 'Visita',
                     'general' : 'Gral.',
                     'corr' : 'Correg.',
                     'perc' : '%',
                     ' p': ' Primeros ',
                     ' c': ' Corte ',
                     'lv' : 'LV',
                     'elo' : 'ELO',
                     'Primer Fácil C': 'Primer Fácil LV C',
                     'Primer Difícil C': 'Primer Difícil LV C'}
    count = 0
    for j in range(len(ligas)):
        print("Leyendo: %s                             " % ligas[j], end = '\t\r')
        df_efecto = pd.read_excel(os.path.join(datadir, ligas[j] + '.xlsx'),
                                  sheet_name = ligas[j])
        cols_categoria = [i for i in df_efecto.columns if 'categoria' in i]
        for col in cols_categoria:
            df_efecto[col] = df_efecto[col].astype('category')
            df_efecto[col].cat.reorder_categories(['regular', 'facil','dificil'], inplace = True)
        if count < 1:
            lista_columnas = [i for i in df_efecto.columns.tolist() if i not in ['equipo','position','Torneo','Pais']
                             and '.' not in i]
#             lista_columnas = [i for i in df_efecto.columns.tolist() if 'perc' in i 
#                               and '.' not in i]
            variables = lista_columnas
            for key, value in dict_replaces.items():
                variables = [i.replace(key,value) for i in variables]
            dict_columns = dict(zip(variables, lista_columnas))
            dfs_efecto = []
            count = count + 1
        dfs_efecto.append(df_efecto)
    return dfs_efecto, variables, dict_columns

In [22]:
datadir = os.path.join(os.path.pardir,'datos','regresiones')
ligas = ['Alemania','Espana','Francia','Inglaterra','Italia']
dfs_efecto, variables, dict_columns = leer_efectos(datadir,
                                                   ligas)

['equipo',
 'position_final',
 'points_final',
 'position_mitad',
 'points_mitad',
 'position_l_final',
 'points_l_final',
 'position_l_mitad',
 'points_l_mitad',
 'position_v_final',
 'points_v_final',
 'position_v_mitad',
 'points_v_mitad',
 'Torneo',
 'perc_facil_general_p4_c3',
 'perc_dificil_general_p4_c3',
 'facil_elo',
 'perc_facil_elo_p4_c3',
 'dificil_elo',
 'perc_dificil_elo_p4_c3',
 'perc_facil_lv_p4_c3',
 'perc_dificil_lv_p4_c3',
 'prom_posicion_general_p4',
 'prom_posicion_lv_p4',
 'ronda_primer_facil_general_c3',
 'ronda_primer_dificil_general_c3',
 'ronda_primer_facil_lv_c3',
 'ronda_primer_dificil_lv_c3',
 'categoria_general_c3',
 'categoria_local_c3',
 'categoria_visita_c3',
 'categoria_elo_c3',
 'perc_facil_general_p5_c3',
 'perc_dificil_general_p5_c3',
 'perc_facil_elo_p5_c3',
 'perc_dificil_elo_p5_c3',
 'perc_facil_lv_p5_c3',
 'perc_dificil_lv_p5_c3',
 'prom_posicion_general_p5',
 'prom_posicion_lv_p5',
 'perc_facil_general_p6_c3',
 'perc_dificil_general_p6_c3',
 'p

In [23]:
def significancia_variables(mod,var,replace_spaces = True, std_err = True):
    coef = "%.3f" % (np.round(mod.params[var], decimals = 3))
    se = "%.3f" % (np.round(mod.bse[var], decimals = 3))
    if mod.params[var] > 0:
        output = '~' + str(coef)                 
    else:
        output = str(coef)
    if std_err:
        output = output + ' (' + str(se) + ')'   
    pval = np.round(mod.pvalues[var], 3)
    if pval < 0.01:
        output = output + '***'
    elif pval < 0.05 and pval >= 0.01:
        output = output + '**~'
    elif pval < 0.1 and pval >=0.05:
        output = output + '*~~'
    else:
        output = output + '~~~'
    if replace_spaces:
        output = output.replace("~"," ")
    return output

## ANOVA de 1 vía para posiciones

La idea de esto es ver si hay diferencias en términos de posición entre los equipos que son asignados como fáciles o difíciles

In [27]:
def anova_1via_categorias(datadir,ligas):
    dfs_efecto, variables, dict_columns = leer_efectos(datadir = datadir,
                                                       ligas = ligas)
    reverse_dict = inv_map = {v: k for k, v in dict_columns.items()}
    ys = [i for i in dfs_efecto[0].columns.tolist() if 'position' in i or 'points' in i]
    dfs_resumen = []
    for y in ys:
        print(y)
        formula = '%s ~ C(equipo)' % y
        data = []
        for i in range(len(ligas)):
            resultados = []
            liga = ligas[i]
            df_efecto = dfs_efecto[i]
            if i < 1:
                cols_categorias = [j for j in df_efecto.columns if 'categoria' in j]
                columna_var = []
                for col in cols_categorias:
                    for cat in ['facil','dificil']:
                        columna_var.append('%s %s' % (reverse_dict[col], cat))  
                data.append(columna_var)
            for col in cols_categorias:
                for cat in ['facil','dificil']:
                    model = ols(formula, data = df_efecto[df_efecto[col] == cat]).fit()
                    aov_table = sm.stats.anova_lm(model, typ=2)
                    resultados.append(np.round(aov_table['PR(>F)'][0], decimals = 3))
            data.append(resultados)
        colnames = ['Categoria'] + ligas
        data_dict = dict(zip(colnames, data))
        df_resumen = pd.DataFrame(data_dict)
        dfs_resumen.append(df_resumen)

    return dfs_resumen

In [39]:
df_test = pd.read_excel(os.path.join(datadir, 'Inglaterra.xlsx'))

In [40]:
df_test = df_test[['equipo', 'Torneo'] + 
                  [i for i in df_test.columns if 'position' in i or 'points' in i] + 
                  [j for j in df_test.columns if 'categoria' in j]]

In [41]:
df_test[['equipo', 'Torneo'] + [i for i in df_test.columns if 'elo' in i]]

,equipo,Torneo,categoria_elo_c3,categoria_elo_c4,categoria_elo_c5
0,Manchester United,Premier League 1993/1994,dificil,dificil,dificil
1,Blackburn Rovers,Premier League 1993/1994,regular,regular,regular
2,Newcastle United,Premier League 1993/1994,facil,facil,facil
3,Arsenal FC,Premier League 1993/1994,dificil,dificil,dificil
4,Leeds United,Premier League 1993/1994,regular,regular,regular
5,Wimbledon FC,Premier League 1993/1994,regular,regular,regular
6,Sheffield Wednesday,Premier League 1993/1994,regular,regular,regular
7,Liverpool FC,Premier League 1993/1994,regular,regular,dificil
8,Queens Park Rangers,Premier League 1993/1994,regular,dificil,dificil
9,Aston Villa,Premier League 1993/1994,dificil,dificil,dificil


In [42]:
dfs_resumen = anova_1via_categorias(datadir,ligas)

position_finala                             	   	
points_final
position_mitad
points_mitad
position_l_final
points_l_final
position_l_mitad
points_l_mitad
position_v_final
points_v_final
position_v_mitad
points_v_mitad


In [46]:
dfs_resumen[4]

,Categoria,Alemania,Espana,Francia,Inglaterra,Italia
0,Categoría Gral. Corte 3 facil,0.098,0.204,0.170,0.811,0.106
1,Categoría Gral. Corte 3 dificil,0.001,0.000,0.016,0.000,0.490
2,Categoría Local Corte 3 facil,0.446,0.451,0.011,0.274,0.002
3,Categoría Local Corte 3 dificil,0.002,0.000,0.014,0.000,0.000
4,Categoría Visita Corte 3 facil,0.071,0.021,0.358,0.246,0.154
5,Categoría Visita Corte 3 dificil,0.001,0.000,0.045,0.000,0.365
6,Categoría ELO Corte 3 facil,0.013,0.431,0.270,0.061,0.447
7,Categoría ELO Corte 3 dificil,0.001,0.002,0.159,0.003,0.200
8,Categoría Gral. Corte 4 facil,0.059,0.331,0.139,0.756,0.069
9,Categoría Gral. Corte 4 dificil,0.006,0.000,0.005,0.000,0.005


In [5]:
def regresion_categorias(datadir,ligas):
    dfs_efecto, variables, dict_columns = leer_efectos(datadir = datadir,
                                                       ligas = ligas)
    reverse_dict = inv_map = {v: k for k, v in dict_columns.items()}
    formula = 'position ~ equipo + %s'
    data = []
    for i in range(len(ligas)):
        resultados = []
        liga = ligas[i]
        df_efecto = dfs_efecto[i]
        if i < 1:
            cols_categorias = [j for j in df_efecto.columns if 'categoria' in j]
            columna_var = []
            for col in cols_categorias:
                for cat in ['facil','dificil']:
                    columna_var.append('%s %s' % (reverse_dict[col], cat))  
            data.append(columna_var)
        for col in cols_categorias:
            model = ols(formula % col, data = df_efecto).fit(disp = False)
            for cat in ['facil','dificil']:
                var = "%s[T.%s]" % (col,cat)
                resultados.append(significancia_variables(model,var,std_err=False, replace_spaces = False))
        data.append(resultados)
    colnames = ['Categoria'] + ligas
    data_dict = dict(zip(colnames, data))
    df_resumen = pd.DataFrame(data_dict)
    return df_resumen

In [6]:
# #regresion
# datadir = os.path.join(os.pardir,
#                        'Datos',
#                        'Bases construidas',
#                        'Bases main')
# ligas = ['Alemania', 'Chile','Espana','Francia','Inglaterra','Italia']
# df_dif_cat = regresion_categorias(datadir,ligas)

In [7]:
# print(df_dif_cat.sort_values(by=['Categoria'], ascending = True).to_latex(index=False).replace('\\textasciitilde','~'))

In [8]:
# datadir = os.path.join(os.pardir,
#                        'Datos',
#                        'Bases construidas',
#                        'Bases main')
# ligas = ['Alemania', 'Chile','Espana','Francia','Inglaterra','Italia']
# df_dif_cat = anova_1via_categorias(datadir,ligas)

In [9]:
# df_dif_cat = df_dif_cat.sort_values(by=['Categoria'], ascending = True)

In [10]:
# print(df_dif_cat.to_latex(index=False))

## ANOVA de 1 vía para indicadores

In [11]:
def significancia_variables_manual(coef,se,pval):
    coef = "%.3f" % (np.round(coef, decimals = 3))
    se = "%.3f" % (np.round(se, decimals = 3))
    if coef > 0:
        output = '_' + str(coef) + ' (' + str(se) + ')'                    
    else:
        output = str(coef) + ' (' + str(se) + ')'
    pval = np.round(pval, 3)
    if pval < 0.01:
        output = output + '***'
    elif pval < 0.05 and pval >= 0.01:
        output = output + '**_'
    elif pval < 0.1 and pval >=0.05:
        output = output + '*__'
    else:
        output = output + '___'
    output = output.replace("_"," ")
    return output

In [12]:
def anova_1via(datadir,
               archivo_resumen,
               outputdir,
               ligas = ['Alemania', 'Chile','Espana','Francia','Inglaterra','Italia']):
    ti = time()
    dfs_efecto, variables, dict_columns = leer_efectos(datadir = datadir,
                                                       ligas = ligas)
    xl_resumen = pd.ExcelFile(os.path.join(datadir, archivo_resumen))
    sheets_resumen = xl_resumen.sheet_names
    df_resumen = pd.read_excel(xl_resumen, sheet_name = 'Completo')
    for j in range(len(ligas)):
        liga = ligas[j]
        resultados = []
        df_reg = df_resumen.copy()
        for variable in variables:
            var = dict_columns[variable]
            formula = var + ' ~ C(equipo)'
            model = ols(formula,
                        data = dfs_efecto[j]).fit()
            aov_table = sm.stats.anova_lm(model, typ=2)
            resultados.append(np.round(aov_table['PR(>F)'][0], decimals = 3))
        df_reg[liga] = resultados
    df_reg = df_reg[['Variable'] + ligas]
    rows_output = []
    dfs = []
    for sheet in sheets_resumen:
        if sheet != 'Completo':
            df_resumen_aux = pd.read_excel(xl_resumen,
                                       sheet_name = sheet)
            variables_sh = df_resumen_aux['Variable'].tolist()
            df_reg_aux = df_reg[df_reg['Variable'].isin(variables_sh)].reset_index(drop=True)
            row_out = [sheet] + [np.round(p, decimals = 3) for p in df_reg_aux[ligas].mean().tolist()]
            rows_output.append(row_out)
            dfs.append(df_reg_aux)
        else:
            dfs.append(df_reg)
    writer = pd.ExcelWriter(os.path.join(outputdir,  archivo_resumen),
                            engine='xlsxwriter')
    for dataframe, sheet in zip(dfs, sheets_resumen):
        dataframe.to_excel(writer,
                           sheet_name = sheet,
                           index = False,
                           freeze_panes = (1,0))
        worksheet = writer.sheets[sheet]
        for idx, col in enumerate(dataframe):
            series = dataframe[col]
            max_len = max((
                series.astype(str).map(len).max(),
                len(str(series.name))
                )) + 1
            worksheet.set_column(idx, idx, max_len)  # set column width
    writer.save()
    df_out = pd.DataFrame(data = rows_output, columns = ['Indicador'] + ligas)
    tf = time()
    deltat = str(datetime.timedelta(seconds=tf - ti))[:7]
    print('[OK]Ejecución terminada en %s                                                     ' % deltat, end ='\t\r')
    return df_out

In [13]:
# datadir = os.path.join(os.pardir,
#                        'Datos',
#                        'Bases construidas',
#                        'Bases main')
# archivo_resumen = 'Resumen Test F.xlsx'
# outputdir = os.path.join(os.pardir,
#                          'Datos',
#                          'Bases construidas')
# ligas = ['Alemania', 'Chile','Espana','Francia','Inglaterra','Italia']
# df_pvals = anova_1via(datadir,archivo_resumen, datadir)

In [14]:
# df_pvals.to_excel(os.path.join(datadir,'Resumen Test F promedio p-valores.xlsx'), index = False)

## Regresiones considerando posición como variable independiente
### Regresiones considerando un indicador

In [15]:
def resumen_regresiones(datadir,
                        archivo_resumen,
                        outputdir,
                        archivos_reg,
                        formulas,
                        ligas = ['Alemania', 'Chile','Espana','Francia','Inglaterra','Italia']):
    dfs_efecto, variables, dict_columns = leer_efectos(datadir = datadir,
                                                       ligas = ligas)
    xl_resumen = pd.ExcelFile(os.path.join(datadir, archivo_resumen))
    sheets_resumen = xl_resumen.sheet_names
    df_resumen = pd.read_excel(xl_resumen, sheet_name = 'Completo')
    max_len_liga = len(max(ligas, key=len))
    max_len_variable = len(max(variables, key=len))
    ti = time()
    for i in range(len(formulas)):
        df_reg = df_resumen.copy()
        formula = formulas[i]
        archivo_reg = archivos_reg[i]
        print('Creando archivo: %s                                                                      ' % archivo_reg,
              end='\t\r')
        for j in range(len(ligas)):
            resultados = []
            for variable in variables:
                var = dict_columns[variable]
                formula_ = formula + ' + ' + var
                if ligas[j] != 'Liga Random':
                    mod = ols(formula = formula_, data = dfs_efecto[j]).fit()
                    output = significancia_variables(mod,var)
                else:
                    df_random = dfs_efecto[j]
                    torneos = df_random['Torneo'].tolist()
                    coefs = np.zeros(100)
                    ses = np.zeros(100)
                    pvals = np.zeros(100)
                    for k in range(100):
                        torneos_sim = np.random.choice(torneos, 10, replace = False).tolist()
                        df_random_sim = df_random[df_random['Torneo'].isin(torneos_sim)]
                        mod = ols(formula = formula_, data = df_random_sim).fit()
                        coefs[k] = np.round(mod.params[var], decimals = 3)
                        ses[k] = np.round(mod.bse[var], decimals = 3)
                        pvals[k] = np.round(mod.pvalues[var], decimals = 3)
                    coef = "%.3f" % (np.round(np.mean(coefs), decimals = 3))
                    se = "%.3f" % (np.round(np.mean(ses), decimals = 3))
                    pval = np.round(np.mean(pvals), 3)
                    output = significancia_variables_manual(coef,se,pval)
                resultados.append(output)
            df_reg[ligas[j]] = resultados
        df_reg = df_reg[['Variable'] + ligas]
        dfs = []
        for sheet in sheets_resumen:
            if sheet != 'Completo':
                df_resumen_aux = pd.read_excel(xl_resumen,
                                           sheet_name = sheet)
                variables_sh = df_resumen_aux['Variable'].tolist()
                dfs.append(df_reg[df_reg['Variable'].isin(variables_sh)].reset_index(drop=True))
            else:
                dfs.append(df_reg)
        writer = pd.ExcelWriter(os.path.join(outputdir, archivo_reg + '.xlsx'),
                                engine='xlsxwriter')
        for dataframe, sheet in zip(dfs, sheets_resumen):
            dataframe.to_excel(writer,
                               sheet_name = sheet,
                               index = False,
                               freeze_panes = (1,0))
            worksheet = writer.sheets[sheet]
            for idx, col in enumerate(dataframe):
                series = dataframe[col]
                max_len = max((
                    series.astype(str).map(len).max(),
                    len(str(series.name))
                    )) + 1
                worksheet.set_column(idx, idx, max_len)  # set column width
        writer.save()
    tf = time()
    deltat = str(datetime.timedelta(seconds=tf - ti))[:7]
    print('[OK]Ejecución terminada en %s                                                     ' % deltat, end ='\t\r')

In [16]:
# datadir = os.path.join(os.pardir,
#                        'Datos',
#                        'Bases construidas',
#                        'Bases main')
# archivo_resumen = 'Resumen Test F.xlsx'
# formulas = ['position ~ ', 'position ~ equipo', 'position ~ Torneo', 'position ~ equipo + Torneo']
# archivos_reg = ['Resumen Reg. Base', 'Resumen Reg. EF Equipo', 'Resumen Reg. EF Torneo', 'Resumen Reg. EFs Equipo Torneo']
# outputdir = os.path.join(os.pardir,
#                          'Datos',
#                          'Bases construidas',
#                          'Resumen Regresiones 1 variable')
# ligas = ['Alemania', 'Chile','Espana','Francia','Inglaterra','Italia']
# # dfs_efecto, variables, dict_columns = leer_efectos(datadir, archivo_resumen, ligas)
# # dict_columns
# resumen_regresiones(datadir = datadir,
#                     archivo_resumen = archivo_resumen,
#                     outputdir = outputdir,
#                     archivos_reg = archivos_reg,
#                     formulas = formulas,
#                     ligas = ligas)

### Regresiones considerando 1 indicador y heterogeneidad

Versión split: corta la base de datos en función de la dificultad de los equipos

In [17]:
def resumen_regresiones_het_split(datadir,
                                  outputdir,
                                  archivo_resumen,
                                  archivos_reg,
                                  formulas,
                                  ligas = ['Alemania', 'Chile','Espana','Francia','Inglaterra','Italia']):
    ti = time()
    xl_resumen = pd.ExcelFile(os.path.join(datadir, archivo_resumen))
    sheets_pairs = [i for i in xl_resumen.sheet_names if 'Completo' not in i]
    dfs_efecto, variables, dict_columns = leer_efectos(datadir = datadir,
                                                       ligas = ligas)
    for i in range(len(formulas)):
        formula = formulas[i]
        archivo_reg = archivos_reg[i]
        print('Creando archivo: %s                                                                      ' % archivo_reg,
              end='\t\r')
        dfs = []
        for sheet in sheets_pairs:
            df_res = pd.read_excel(xl_resumen, sheet_name = sheet)
            variables_sh = df_res['Variable'].tolist()
            rows_list = []
            for variable in variables_sh:
                corte = variable[-1]
                var = dict_columns[variable]
                if 'Gral.' in sheet:
                    categoria = 'Categoría Gral. Corte %s' % corte
                    cat = dict_columns[categoria]
                    formula_ = formula + ' + ' + var
                    for c in ['facil','regular','dificil']:
                        var_l = [variable + ' General %s' % c]                 
                        for j in range(len(ligas)):
                            df_current = dfs_efecto[j]
                            mod = ols(formula = formula_, data = df_current[df_current[cat] == c]).fit()
                            var_l.append(significancia_variables(mod, var))
                        rows_list.append(var_l)
                else:
                    categorias = ['Categoría Local Corte %s' % corte, 'Categoría Visita Corte %s' % corte]
                    categoria_corta = ['Local','Visita']
                    formula_ = formula + ' + ' +  var
                    for k in range(len(categorias)):
                        categoria = categorias[k]
                        cat = dict_columns[categoria]
                        for c in ['facil','regular','dificil']:
                            var_l = [variable + ' %s %s' % (categoria_corta[k], c)]                 
                            for j in range(len(ligas)):
                                df_current = dfs_efecto[j]
                                mod = ols(formula = formula_, data = df_current[df_current[cat] == c]).fit()
                                var_l.append(significancia_variables(mod, var))
                            rows_list.append(var_l)              
            df_reg = pd.DataFrame(data = rows_list,
                                  columns = ['Variable'] + ligas)
            dfs.append(df_reg)
        writer = pd.ExcelWriter(os.path.join(outputdir, archivo_reg + '.xlsx'),
                                engine='xlsxwriter')
        df_reg2 = pd.concat(dfs, ignore_index = True)
        df_reg2.to_excel(writer,
                         sheet_name = 'Completo',
                         index = False,
                         freeze_panes = (1,0))
        worksheet = writer.sheets['Completo']
        for idx, col in enumerate(df_reg2):
            series = df_reg2[col]
            max_len = max((
                series.astype(str).map(len).max(),
                len(str(series.name))
                ))
            worksheet.set_column(idx, idx, max_len)  # set column width
        for dataframe, sheet in zip(dfs, sheets_pairs):
            dataframe.to_excel(writer,
                               sheet_name = sheet,
                               index = False,
                               freeze_panes = (1,0))
            worksheet = writer.sheets[sheet]
            for idx, col in enumerate(dataframe):
                series = dataframe[col]
                max_len = max((
                    series.astype(str).map(len).max(),
                    len(str(series.name))
                    ))
                worksheet.set_column(idx, idx, max_len)  # set column width
        writer.save()
    tf = time()
    deltat = str(datetime.timedelta(seconds=tf - ti))[:7]
    print('[OK]Ejecución terminada en %s                                                     ' % deltat, end ='\t\r')

In [18]:
# datadir = os.path.join(os.pardir,
#                        'Datos',
#                        'Bases construidas',
#                        'Bases main')
# outputdir = os.path.join(os.pardir,
#                          'Datos',
#                          'Bases construidas',
#                          'Resumen Regresiones 1 variable het split')
# archivo_resumen = 'Resumen Reg. Base.xlsx'
# archivos_reg = ['Resumen Reg. Base',
#                 'Resumen Reg. EF Equipo',
#                 'Resumen Reg. EF Torneo',
#                 'Resumen Reg. EFs Equipo Torneo']
# formulas = ['position ~ ', 'position ~ equipo', 'position ~ Torneo', 'position ~ equipo + Torneo']
# ligas = ['Alemania', 'Chile','Espana','Francia','Inglaterra','Italia']
# resumen_regresiones_het_split(datadir,
#                               outputdir,
#                               archivo_resumen,
#                               archivos_reg,
#                               formulas,
#                               ligas)

Versión full: considera toda la base de datos y utiliza interacciones de variables

In [19]:
def resumen_regresiones_het(datadir,
                            outputdir,
                            archivo_resumen,
                            archivos_reg,
                            formulas,
                            ligas = ['Alemania', 'Chile','Espana','Francia','Inglaterra','Italia']):
    ti = time()
    xl_resumen = pd.ExcelFile(os.path.join(datadir, archivo_resumen))
    sheets_pairs = [i for i in xl_resumen.sheet_names if 'Completo' not in i]
    emparejamiento = ['Gral.', 'LV']

    dfs_efecto, variables, dict_columns = leer_efectos(datadir = datadir,
                                                       ligas = ligas)
    for i in range(len(formulas)):
        formula = formulas[i]
        archivo_reg = archivos_reg[i]
        print('Creando archivo: %s                                                                      ' % archivo_reg,
              end='\t\r')
        dfs = []
        for sheet in sheets_pairs:
            df_res = pd.read_excel(xl_resumen, sheet_name = sheet)
            variables_sh = df_res['Variable'].tolist()
            rows_list = []
            for variable in variables_sh:
                corte = variable[-1]
                var = dict_columns[variable]
                if 'Gral.' in sheet:
                    categoria = 'Categoría Gral. Corte %s' % corte
                    cat = dict_columns[categoria]
                    formula_ = formula + ' + ' + cat + '*' + var
                    varf_l = [variable + '*' + categoria +' Fácil']
                    vard_l = [variable + '*' + categoria +' Difícil']
                    for j in range(len(ligas)):
                        mod = ols(formula = formula_, data = dfs_efecto[j]).fit()
                        varf_l.append(significancia_variables(mod,cat + '[T.facil]:' + var))
                        vard_l.append(significancia_variables(mod,cat + '[T.dificil]:' + var))
                    rows_list.append(varf_l)
                    rows_list.append(vard_l)
                else:
                    categoria1, categoria2 = 'Categoría Local Corte %s' % corte, 'Categoría Visita Corte %s' % corte
                    cat1, cat2 = dict_columns[categoria1], dict_columns[categoria2]
                    formula_ = formula + ' + ' +  cat1 + '*' + var + '+' + cat2 + '*' + var
                    varf1_l = [variable + '*' + categoria1 + ' Fácil']
                    vard1_l = [variable + '*' + categoria1 + ' Difícil']
                    varf2_l = [variable + '*' + categoria2 + ' Fácil']
                    vard2_l = [variable + '*' + categoria2 + ' Difícil']
                    for j in range(len(ligas)):
                        mod = ols(formula = formula_, data = dfs_efecto[j]).fit()
                        varf1_l.append(significancia_variables(mod,cat1 + '[T.facil]:' + var))
                        vard1_l.append(significancia_variables(mod,cat1 + '[T.dificil]:' + var))
                        varf2_l.append(significancia_variables(mod,cat2 + '[T.facil]:' + var))
                        vard2_l.append(significancia_variables(mod,cat2 + '[T.dificil]:' + var))
                    rows_list.append(varf1_l)
                    rows_list.append(vard1_l)
                    rows_list.append(varf2_l)
                    rows_list.append(vard2_l)
                    
                
            df_reg = pd.DataFrame(data = rows_list,
                                  columns = ['Variable'] + ligas)
            dfs.append(df_reg)
        writer = pd.ExcelWriter(os.path.join(outputdir, archivo_reg + '.xlsx'),
                                engine='xlsxwriter')
        df_reg2 = pd.concat(dfs, ignore_index = True)
        df_reg2.to_excel(writer,
                         sheet_name = 'Completo',
                         index = False,
                         freeze_panes = (1,0))
        worksheet = writer.sheets['Completo']
        for idx, col in enumerate(df_reg2):
            series = df_reg2[col]
            max_len = max((
                series.astype(str).map(len).max(),
                len(str(series.name))
                ))
            worksheet.set_column(idx, idx, max_len)  # set column width
        for dataframe, sheet in zip(dfs, sheets_pairs):
            dataframe.to_excel(writer,
                               sheet_name = sheet,
                               index = False,
                               freeze_panes = (1,0))
            worksheet = writer.sheets[sheet]
            for idx, col in enumerate(dataframe):
                series = dataframe[col]
                max_len = max((
                    series.astype(str).map(len).max(),
                    len(str(series.name))
                    ))
                worksheet.set_column(idx, idx, max_len)  # set column width
        writer.save()
    tf = time()
    deltat = str(datetime.timedelta(seconds=tf - ti))[:7]
    print('[OK]Ejecución terminada en %s                                                     ' % deltat, end ='\t\r')

In [20]:
# datadir = os.path.join(os.pardir,
#                        'Datos',
#                        'Bases construidas',
#                        'Bases main')
# outputdir = os.path.join(os.pardir,
#                          'Datos',
#                          'Bases construidas',
#                          'Resumen Regresiones 1 variable het')
# archivo_resumen = 'Resumen Reg. Base.xlsx'
# archivos_reg = ['Resumen Reg. Base',
#                 'Resumen Reg. EF Equipo',
#                 'Resumen Reg. EF Torneo',
#                 'Resumen Reg. EFs Equipo Torneo']
# formulas = ['position ~ ', 'position ~ equipo', 'position ~ Torneo', 'position ~ equipo + Torneo']
# ligas = ['Alemania', 'Chile','Espana','Francia','Inglaterra','Italia']
# resumen_regresiones_het(datadir,
#                         outputdir,
#                         archivo_resumen,
#                         archivos_reg,
#                         formulas,
#                         ligas)

### Regresiones considerando 2 indicadores

In [21]:
def resumen_regresiones_pares(datadir,
                              outputdir,
                              archivo_resumen,
                              archivos_reg,
                              formulas,
                              pairs,
                              sheets_pairs,
                              cortes = [3,4,5],
                              rondas = [4,5,6,7],
                              ligas = ['Alemania', 'Chile','Espana','Francia','Inglaterra','Italia']):
    ti = time()
    dfs_efecto, variables, dict_columns = leer_efectos(datadir = datadir,
                                                       ligas = ligas)
    for i in range(len(formulas)):
        formula = formulas[i]
        archivo_reg = archivos_reg[i]
        print('Creando archivo: %s                                                                      ' % archivo_reg,
              end='\t\r')
        dfs = []
        for pair in pairs:
            variable1, variable2 = pair[0], pair[1]
            rows_list = []
            for corte in cortes:
                if "Primeros" not in variable1:
                    variable_1, variable_2 = variable1 % corte, variable2 % corte
                    var1, var2 = dict_columns[variable_1], dict_columns[variable_2]
                    formula_ = formula + ' + ' + var1 + ' + ' + var2
                    var1_l, var2_l = [variable_1], [variable_2]
                    for j in range(len(ligas)):
                        mod = ols(formula = formula_, data = dfs_efecto[j]).fit()
                        var1_l.append(significancia_variables(mod,var1))
                        var2_l.append(significancia_variables(mod,var2))
                    rows_list.append(var1_l)
                    rows_list.append(var2_l)
                else:
                    for ronda in rondas:
                        variable_1, variable_2 = variable1 % (ronda, corte), variable2 % (ronda, corte)
                        var1, var2 = dict_columns[variable_1], dict_columns[variable_2]
                        formula_ = formula + ' + ' + var1 + ' + ' + var2
                        var1_l, var2_l = [variable_1], [variable_2]
                        for j in range(len(ligas)):
                            mod = ols(formula = formula_, data = dfs_efecto[j]).fit()
                            var1_l.append(significancia_variables(mod,var1))
                            var2_l.append(significancia_variables(mod,var2))
                        rows_list.append(var1_l)
                        rows_list.append(var2_l)
            df_reg = pd.DataFrame(data = rows_list,
                                  columns = ['Variable'] + ligas)
            dfs.append(df_reg)
        writer = pd.ExcelWriter(os.path.join(outputdir, archivo_reg + '.xlsx'),
                                engine='xlsxwriter')
        df_reg2 = pd.concat(dfs, ignore_index = True)
        df_reg2.to_excel(writer,
                         sheet_name = 'Completo',
                         index = False,
                         freeze_panes = (1,0))
        worksheet = writer.sheets['Completo']
        for idx, col in enumerate(df_reg2):
            series = df_reg2[col]
            max_len = max((
                series.astype(str).map(len).max(),
                len(str(series.name))
                )) + 1
            worksheet.set_column(idx, idx, max_len)  # set column width
        for dataframe, sheet in zip(dfs, sheets_pairs):
            dataframe.to_excel(writer,
                               sheet_name = sheet,
                               index = False,
                               freeze_panes = (1,0))
            worksheet = writer.sheets[sheet]
            for idx, col in enumerate(dataframe):
                series = dataframe[col]
                max_len = max((
                    series.astype(str).map(len).max(),
                    len(str(series.name))
                    )) + 1
                worksheet.set_column(idx, idx, max_len)  # set column width
        writer.save()
    tf = time()
    deltat = str(datetime.timedelta(seconds=tf - ti))[:7]
    print('[OK]Ejecución terminada en %s                                                     ' % deltat, end ='\t\r')

In [22]:
# datadir = os.path.join(os.pardir,
#                        'Datos',
#                        'Bases construidas',
#                        'Bases main')
# archivo_resumen = 'Resumen Test F.xlsx'
# formulas = ['position ~ ', 'position ~ equipo', 'position ~ Torneo', 'position ~ equipo + Torneo']
# archivos_reg = ['Resumen Reg. Base', 'Resumen Reg. EF Equipo', 'Resumen Reg. EF Torneo', 'Resumen Reg. EFs Equipo Torneo']
# outputdir = os.path.join(os.pardir,
#                          'Datos',
#                          'Bases construidas',
#                          'Resumen Regresiones 2 variables')
# # pares de variables
# pairs = [['%% Difícil Gral. Primeros %s Corte %s', '%% Fácil Gral. Primeros %s Corte %s'],
#          ['%% Difícil Gral. Correg. Primeros %s Corte %s', '%% Fácil Gral. Correg. Primeros %s Corte %s'],
#          ['%% Difícil LV Primeros %s Corte %s', '%% Fácil LV Primeros %s Corte %s'],
#          ['%% Difícil LV Correg. Primeros %s Corte %s', '%% Fácil LV Correg. Primeros %s Corte %s'],
#          ['Ronda Primer Difícil Gral. Corte %s', 'Ronda Primer Fácil Gral. Corte %s'],
#          ['Ronda Primer Difícil LV Corte %s', 'Ronda Primer Fácil LV Corte %s']]
# sheets_pairs = ['% Gral.', '% Gral. Correg.', '% LV', '% LV Correg.', 'Ronda Gral.', 'Ronda LV']
# resumen_regresiones_pares(datadir = datadir,
#                           outputdir = outputdir,
#                           archivo_resumen = archivo_resumen,
#                           archivos_reg = archivos_reg,
#                           formulas = formulas,
#                           pairs = pairs,
#                           sheets_pairs = sheets_pairs)

### Regresión considerando 2 indicadores y heterogeneidad

Versión split: corta la base de datos según categorías

In [23]:
def constructor_reg_2vars_het_split(sheet,ligas,dict_columns,formula,dfs_efecto,variable_1,variable_2,corte):
    var1, var2 = dict_columns[variable_1], dict_columns[variable_2]   
    output = []
    formula_ = formula + '+ %s + %s' % (var1, var2)
    if 'Gral.' in sheet:
        categoria = 'Categoría Gral. Corte %s' % corte
        cat = dict_columns[categoria]

        for c in ['facil','regular','dificil']:
            var1_l = [variable_1 + ' General %s' % c]
            var2_l = [variable_2 + ' General %s' % c]
            for j in range(len(ligas)):
                df_current = dfs_efecto[j]
                mod = ols(formula = formula_, data = df_current[df_current[cat] == c]).fit()
                var1_l.append(significancia_variables(mod, var1))
                var2_l.append(significancia_variables(mod, var2))
            output.append(var1_l)
            output.append(var2_l)
    else:
        categorias = ['Categoría Local Corte %s' % corte, 'Categoría Visita Corte %s' % corte]
        categoria_corta = ['Local','Visita']
        for k in range(len(categorias)):
            categoria = categorias[k]
            cat = dict_columns[categoria]
            for c in ['facil','regular','dificil']:
                var1_l = [variable_1 + ' %s %s' % (categoria_corta[k], c)]
                var2_l = [variable_2 + ' %s %s' % (categoria_corta[k], c)]  
                for j in range(len(ligas)):
                    df_current = dfs_efecto[j]
                    mod = ols(formula = formula_, data = df_current[df_current[cat] == c]).fit()
                    var1_l.append(significancia_variables(mod, var1))
                    var2_l.append(significancia_variables(mod, var2))
                output.append(var1_l)
                output.append(var2_l)
    return output

In [24]:
def resumen_regresiones_pares_het_split(datadir,
                                        outputdir,
                                        archivo_resumen,
                                        archivos_reg,
                                        formulas,
                                        pairs,
                                        sheets_pairs,
                                        cortes = [3,4,5],
                                        rondas = [4,5,6,7],
                                        ligas = ['Alemania', 'Chile','Espana','Francia','Inglaterra','Italia']):
    ti = time()
    dfs_efecto, variables, dict_columns = leer_efectos(datadir = datadir,
                                                       ligas = ligas)
    for i in range(len(formulas)):
        formula = formulas[i]
        archivo_reg = archivos_reg[i]
        print('Creando archivo: %s                                                                      ' % archivo_reg,
              end='\t\r')
        dfs = []
        for k in range(len(pairs)):
            pair = pairs[k]
            variable1, variable2 = pair[0], pair[1]
            rows_list = []
            sheet = sheets_pairs[k]
            for corte in cortes:
                if "Primeros" not in variable1:
                    variable_1, variable_2 = variable1 % corte, variable2 % corte
                    aux = constructor_reg_2vars_het_split(sheet,
                                                          ligas,
                                                          dict_columns,
                                                          formula,
                                                          dfs_efecto,
                                                          variable_1,
                                                          variable_2,
                                                          corte)
                    rows_list = rows_list + aux
                else:
                    for ronda in rondas:
                        variable_1, variable_2 = variable1 % (ronda, corte), variable2 % (ronda, corte)
                        aux = constructor_reg_2vars_het_split(sheet,
                                                              ligas,
                                                              dict_columns,
                                                              formula,
                                                              dfs_efecto,
                                                              variable_1,
                                                              variable_2,
                                                              corte)
                        rows_list = rows_list + aux
            df_reg = pd.DataFrame(data = rows_list,
                                  columns = ['Variable'] + ligas)
            dfs.append(df_reg)
        writer = pd.ExcelWriter(os.path.join(outputdir, archivo_reg + '.xlsx'),
                                engine='xlsxwriter')
        df_reg2 = pd.concat(dfs, ignore_index = True)
        df_reg2.to_excel(writer,
                         sheet_name = 'Completo',
                         index = False,
                         freeze_panes = (1,0))
        worksheet = writer.sheets['Completo']
        for idx, col in enumerate(df_reg2):
            series = df_reg2[col]
            max_len = max((
                series.astype(str).map(len).max(),
                len(str(series.name))
                ))
            worksheet.set_column(idx, idx, max_len)  # set column width
        for dataframe, sheet in zip(dfs, sheets_pairs):
            dataframe.to_excel(writer,
                               sheet_name = sheet,
                               index = False,
                               freeze_panes = (1,0))
            worksheet = writer.sheets[sheet]
            for idx, col in enumerate(dataframe):
                series = dataframe[col]
                max_len = max((
                    series.astype(str).map(len).max(),
                    len(str(series.name))
                    ))
                worksheet.set_column(idx, idx, max_len)  # set column width
        writer.save()
    tf = time()
    deltat = str(datetime.timedelta(seconds=tf - ti))[:7]
    print('[OK]Ejecución terminada en %s                                                     ' % deltat, end ='\t\r')

In [25]:
# datadir = os.path.join(os.pardir,
#                        'Datos',
#                        'Bases construidas',
#                        'Bases main')
# archivo_resumen = 'Resumen Test F.xlsx'
# formulas = ['position ~ ', 'position ~ equipo', 'position ~ Torneo', 'position ~ equipo + Torneo']
# archivos_reg = ['Resumen Reg. Base', 'Resumen Reg. EF Equipo', 'Resumen Reg. EF Torneo', 'Resumen Reg. EFs Equipo Torneo']
# outputdir = os.path.join(os.pardir,
#                          'Datos',
#                          'Bases construidas',
#                          'Resumen Regresiones 2 variables het split')
# # pares de variables
# pairs = [['%% Difícil Gral. Primeros %s Corte %s', '%% Fácil Gral. Primeros %s Corte %s'],
#          ['%% Difícil Gral. Correg. Primeros %s Corte %s', '%% Fácil Gral. Correg. Primeros %s Corte %s'],
#          ['%% Difícil LV Primeros %s Corte %s', '%% Fácil LV Primeros %s Corte %s'],
#          ['%% Difícil LV Correg. Primeros %s Corte %s', '%% Fácil LV Correg. Primeros %s Corte %s'],
#          ['Ronda Primer Difícil Gral. Corte %s', 'Ronda Primer Fácil Gral. Corte %s'],
#          ['Ronda Primer Difícil LV Corte %s', 'Ronda Primer Fácil LV Corte %s']]
# sheets_pairs = ['% Gral.', '% Gral. Correg.', '% LV', '% LV Correg.', 'Ronda Gral.', 'Ronda LV']
# resumen_regresiones_pares_het_split(datadir = datadir,
#                                     outputdir = outputdir,
#                                     archivo_resumen = archivo_resumen,
#                                     archivos_reg = archivos_reg,
#                                     formulas = formulas,
#                                     pairs = pairs,
#                                     sheets_pairs = sheets_pairs)

In [26]:
def constructor_reg_2vars_het(sheet,ligas,dict_columns,formula,dfs_efecto,variable_1,variable_2,corte):
    var1, var2 = dict_columns[variable_1], dict_columns[variable_2]   
    output = []
    if 'Gral.' in sheet:
        categoria = 'Categoría Gral. Corte %s' % corte
        cat = dict_columns[categoria]
        formula_ = formula + ' + ' + cat + '*' + var1  + ' + ' + cat + '*' + var2
        var1f_l = [variable_1 + '*' + categoria +' Fácil']
        var1d_l = [variable_1 + '*' + categoria +' Difícil']
        var2f_l = [variable_2 + '*' + categoria +' Fácil']
        var2d_l = [variable_2 + '*' + categoria +' Difícil']
        for j in range(len(ligas)):
            mod = ols(formula = formula_, data = dfs_efecto[j]).fit()
            var1f_l.append(significancia_variables(mod,cat + '[T.facil]:' + var1))
            var1d_l.append(significancia_variables(mod,cat + '[T.dificil]:' + var1))
            var2f_l.append(significancia_variables(mod,cat + '[T.facil]:' + var2))
            var2d_l.append(significancia_variables(mod,cat + '[T.dificil]:' + var2))
        output.append(var1f_l)
        output.append(var1d_l)
        output.append(var2f_l)
        output.append(var2d_l)
    else:
        categoria1, categoria2 = 'Categoría Local Corte %s' % corte, 'Categoría Visita Corte %s' % corte
        cat1, cat2 = dict_columns[categoria1], dict_columns[categoria2]
        formula_ = formula + ' + ' +  cat1 + '*' + var1 + '+' + cat2 + '*' + var1 
        formula_ = formula_ + ' + ' +  cat1 + '*' + var2 + '+' + cat2 + '*' + var2 
        var1f1_l = [variable_1 + '*' + categoria1 + ' Fácil']
        var1d1_l = [variable_1 + '*' + categoria1 + ' Difícil']
        var1f2_l = [variable_1 + '*' + categoria2 + ' Fácil']
        var1d2_l = [variable_1 + '*' + categoria2 + ' Difícil']
        var2f1_l = [variable_2 + '*' + categoria1 + ' Fácil']
        var2d1_l = [variable_2 + '*' + categoria1 + ' Difícil']
        var2f2_l = [variable_2 + '*' + categoria2 + ' Fácil']
        var2d2_l = [variable_2 + '*' + categoria2 + ' Difícil']
        for j in range(len(ligas)):
            mod = ols(formula = formula_, data = dfs_efecto[j]).fit()
            var1f1_l.append(significancia_variables(mod,cat1 + '[T.facil]:' + var1))
            var1d1_l.append(significancia_variables(mod,cat1 + '[T.dificil]:' + var1))
            var1f2_l.append(significancia_variables(mod,cat2 + '[T.facil]:' + var1))
            var1d2_l.append(significancia_variables(mod,cat2 + '[T.dificil]:' + var1))
            var2f1_l.append(significancia_variables(mod,cat1 + '[T.facil]:' + var2))
            var2d1_l.append(significancia_variables(mod,cat1 + '[T.dificil]:' + var2))
            var2f2_l.append(significancia_variables(mod,cat2 + '[T.facil]:' + var2))
            var2d2_l.append(significancia_variables(mod,cat2 + '[T.dificil]:' + var2))
        output.append(var1f1_l)
        output.append(var1d1_l)
        output.append(var1f2_l)
        output.append(var1d2_l)
        output.append(var2f1_l)
        output.append(var2d1_l)
        output.append(var2f2_l)
        output.append(var2d2_l)
    return output

In [27]:
def resumen_regresiones_pares_het(datadir,
                                  outputdir,
                                  archivo_resumen,
                                  archivos_reg,
                                  formulas,
                                  pairs,
                                  sheets_pairs,
                                  cortes = [3,4,5],
                                  rondas = [4,5,6,7],
                                  ligas = ['Alemania', 'Chile','Espana','Francia','Inglaterra','Italia']):
    ti = time()
    dfs_efecto, variables, dict_columns = leer_efectos(datadir = datadir,
                                                       ligas = ligas)
    for i in range(len(formulas)):
        formula = formulas[i]
        archivo_reg = archivos_reg[i]
        print('Creando archivo: %s                                                                      ' % archivo_reg,
              end='\t\r')
        dfs = []
        for k in range(len(pairs)):
            pair = pairs[k]
            variable1, variable2 = pair[0], pair[1]
            rows_list = []
            sheet = sheets_pairs[k]
            for corte in cortes:
                if "Primeros" not in variable1:
                    variable_1, variable_2 = variable1 % corte, variable2 % corte
                    aux = constructor_reg_2vars_het(sheet,
                                                    ligas,
                                                    dict_columns,
                                                    formula,
                                                    dfs_efecto,
                                                    variable_1,
                                                    variable_2,
                                                    corte)
                    rows_list = rows_list + aux
                else:
                    for ronda in rondas:
                        variable_1, variable_2 = variable1 % (ronda, corte), variable2 % (ronda, corte)
                        aux = constructor_reg_2vars_het(sheet,
                                                        ligas,
                                                        dict_columns,
                                                        formula,
                                                        dfs_efecto,
                                                        variable_1,
                                                        variable_2,
                                                        corte)
                        rows_list = rows_list + aux
            df_reg = pd.DataFrame(data = rows_list,
                                  columns = ['Variable'] + ligas)
            dfs.append(df_reg)
        writer = pd.ExcelWriter(os.path.join(outputdir, archivo_reg + '.xlsx'),
                                engine='xlsxwriter')
        df_reg2 = pd.concat(dfs, ignore_index = True)
        df_reg2.to_excel(writer,
                         sheet_name = 'Completo',
                         index = False,
                         freeze_panes = (1,0))
        worksheet = writer.sheets['Completo']
        for idx, col in enumerate(df_reg2):
            series = df_reg2[col]
            max_len = max((
                series.astype(str).map(len).max(),
                len(str(series.name))
                ))
            worksheet.set_column(idx, idx, max_len)  # set column width
        for dataframe, sheet in zip(dfs, sheets_pairs):
            dataframe.to_excel(writer,
                               sheet_name = sheet,
                               index = False,
                               freeze_panes = (1,0))
            worksheet = writer.sheets[sheet]
            for idx, col in enumerate(dataframe):
                series = dataframe[col]
                max_len = max((
                    series.astype(str).map(len).max(),
                    len(str(series.name))
                    ))
                worksheet.set_column(idx, idx, max_len)  # set column width
        writer.save()
    tf = time()
    deltat = str(datetime.timedelta(seconds=tf - ti))[:7]
    print('[OK]Ejecución terminada en %s                                                     ' % deltat, end ='\t\r')

In [28]:
# datadir = os.path.join(os.pardir,
#                        'Datos',
#                        'Bases construidas',
#                        'Bases main')
# archivo_resumen = 'Resumen Test F.xlsx'
# formulas = ['position ~ ', 'position ~ equipo', 'position ~ Torneo', 'position ~ equipo + Torneo']
# archivos_reg = ['Resumen Reg. Base', 'Resumen Reg. EF Equipo', 'Resumen Reg. EF Torneo', 'Resumen Reg. EFs Equipo Torneo']
# outputdir = os.path.join(os.pardir,
#                          'Datos',
#                          'Bases construidas',
#                          'Resumen Regresiones 2 variables het')
# # pares de variables
# pairs = [['%% Difícil Gral. Primeros %s Corte %s', '%% Fácil Gral. Primeros %s Corte %s'],
#          ['%% Difícil Gral. Correg. Primeros %s Corte %s', '%% Fácil Gral. Correg. Primeros %s Corte %s'],
#          ['%% Difícil LV Primeros %s Corte %s', '%% Fácil LV Primeros %s Corte %s'],
#          ['%% Difícil LV Correg. Primeros %s Corte %s', '%% Fácil LV Correg. Primeros %s Corte %s'],
#          ['Ronda Primer Difícil Gral. Corte %s', 'Ronda Primer Fácil Gral. Corte %s'],
#          ['Ronda Primer Difícil LV Corte %s', 'Ronda Primer Fácil LV Corte %s']]
# sheets_pairs = ['% Gral.', '% Gral. Correg.', '% LV', '% LV Correg.', 'Ronda Gral.', 'Ronda LV']
# resumen_regresiones_pares_het(datadir,
#                               outputdir,
#                               archivo_resumen,
#                               archivos_reg,
#                               formulas,
#                               pairs,
#                               sheets_pairs,
#                               cortes = [3,4,5],
#                               rondas = [4,5,6,7],
#                               ligas = ['Alemania', 'Chile','Espana','Francia','Inglaterra','Italia'])

## Gráficos para incluir en la tesis
Ligas a considerar: 
- Premier League 
- Serie A?

### Regresión con 1 indicador

In [29]:
def GraficoCoeficientes1VarLR(datadir, archivo, formula, variable, conf = 0.05):
    dictreplaces = {'perc': 'Frac. partidos', 'facil': 'fáciles', 'dificil': 'dificiles','_':' '}
    cint = str(int((1-conf)*100)) + '%'
    title = variable.split('_p')[0]
    for word, replace in dictreplaces.items():
        title = title.replace(word, replace)
    title = 'Efecto indicador ' + title + ' e intervalo de confianza al ' + cint + ' - ' + archivo.replace('.xlsx','')
    df_reg = pd.read_excel(os.path.join(datadir, archivo))
    rl = []
    coefs = []
    lb = []
    ub = []
    for r in [4,5,6,7]:
        for l in [3,4,5]:
            var = variable % (r,l)
            rl.append("$r = %s, l = %s$" % (r,l))
            formula_ = formula % var
            mod = ols(formula = formula_, data = df_reg).fit()
            coefs.append(mod.params[var])
            ci = mod.conf_int(alpha=conf)
            lb.append(ci.loc[var,0])
            ub.append(ci.loc[var,1])
    df_coef = pd.DataFrame(data = {'LB':lb, 'Beta' : coefs, 'UB':ub}, index = rl)
#     fig = plt.figure()
#     ax1=fig.add_subplot(311)
#     ax1.plot(df_coef.index.tolist(), df_coef.Beta.tolist(), color = 'g' ,label='Beta')
#     ax1.plot(df_coef.index.tolist(), df_coef.LB.tolist(),linestyle = ':', color = 'r' ,label='LB 95%')
#     ax1.plot(df_coef.index.tolist(), df_coef.UB.tolist(),linestyle = ':', color = 'b' ,label='UB 95%')
#     ax2=fig.add_subplot(312)
    fig, axs = plt.subplots(figsize=(15,5))
    axs.plot(df_coef.index.tolist(), df_coef.Beta.tolist(), color = 'g' ,label='Beta')
    axs.plot(df_coef.index.tolist(), df_coef.LB.tolist(),linestyle = ':', color = 'r' ,label='LB %s' % cint)
    axs.plot(df_coef.index.tolist(), df_coef.UB.tolist(),linestyle = ':', color = 'b' ,label='UB %s' % cint)
    axs.axhline(y=0, color='k')
    axs.set_title(title)
    axs.set_xlabel('Valores de $r$ y $l$')
    axs.set_ylabel(r'Magnitud efecto ($\beta$)')
    axs.legend()
    plt.text(1,
             -7,
             r'$y_{it} = \beta_0 + \alpha_i + \gamma_t + \beta_1 x_{it} + \varepsilon_{it}$',
             fontdict={'size':16})
    plt.xticks(rotation=90)

In [30]:
def ConstructorCoeficientes1Var(df_reg, variable, formula, conf):
    rl = []
    coefs = []
    lb = []
    ub = []
    for r in [4,5,6,7]:
        for l in [3,4,5]:
            var = variable % (r,l)
            rl.append("$r = %s, l = %s$" % (r,l))
            formula_ = formula % var
            mod = ols(formula = formula_, data = df_reg).fit(cov_type = 'cluster',
                                                             cov_kwds={'groups': df_reg['equipo']})
            coefs.append(mod.params[var])
            ci = mod.conf_int(alpha=conf)
            lb.append(ci.loc[var,0])
            ub.append(ci.loc[var,1])
    df_coef = pd.DataFrame(data = {'LB':lb, 'Beta' : coefs, 'UB':ub}, index = rl)
    return df_coef

In [31]:
def GraficoCoeficientes1VarLR2(datadir, archivo, formulas, textformulas, variable, conf = 0.05):
    dictreplaces = {'perc': 'Frac. partidos', 'facil': 'fáciles', 'dificil': 'dificiles','_':' '}
    cint = str(int((1-conf)*100)) + '%'
    title = variable.split('_p')[0]
    for word, replace in dictreplaces.items():
        title = title.replace(word, replace)
    title = 'Efecto indicador ' + title + ' e intervalo de confianza al ' + cint + ' - ' + archivo.replace('.xlsx','')
    df_reg = pd.read_excel(os.path.join(datadir, archivo))
    fig, axs = plt.subplots(figsize=(10,10), nrows = len(formulas), sharex= True)
    plt.suptitle(title)
    for i in range(len(formulas)):
        formula = formulas[i]
        textformula = textformulas[i]
        ax = axs[i]
        df_coef = ConstructorCoeficientes1Var(df_reg, variable, formula, conf)
        ax.plot(df_coef.index.tolist(), df_coef.Beta.tolist(), color = 'g' ,label='Beta')
        ax.plot(df_coef.index.tolist(), df_coef.LB.tolist(),linestyle = ':', color = 'r' ,label='LB %s' % cint)
        ax.plot(df_coef.index.tolist(), df_coef.UB.tolist(),linestyle = ':', color = 'b' ,label='UB %s' % cint)
        ax.axhline(y=0, color='k')
        ax.set_ylabel(r'Magnitud efecto ($\beta$)')
        ax.legend()
        ax.set_title(textformula)
#         ax.text(1,
#                 df_coef.LB.min() + 1,
#                 textformula,
#                 fontdict={'size':16})
        if i == len(formulas) - 1:
            ax.set_xlabel('Valores de $r$ y $l$')
    fig.tight_layout()
    fig.subplots_adjust(top=0.92)
    plt.xticks(rotation=90)
    fig.savefig(os.path.join(os.path.pardir,'Graficos','Frac. fáciles 1 var.png'), bbox_inches = "tight", dpi = 250)

In [32]:
# datadir = os.path.join(os.pardir,
#                        'Datos',
#                        'Bases construidas',
#                        'Bases main',
#                        'Bases efecto')
# archivo = 'Inglaterra.xlsx'
# formulas = ['position ~ %s',
#             'position ~ equipo + %s',
#             'position~ Torneo + %s',
#             'position ~ equipo + Torneo + %s']
# textformulas = [r'$y_{it} =  \beta_0 + \beta_1 x_{it} + \varepsilon_{it}$',
#                 r'$y_{it} =  \beta_0 + \beta_1 x_{it} + \alpha_i + \varepsilon_{it}$',
#                 r'$y_{it} = \beta_0 + \beta_1 x_{it} + \gamma_t + \varepsilon_{it}$',
#                 r'$y_{it} = \beta_0 + \beta_1 x_{it} + \alpha_i + \gamma_t + \varepsilon_{it}$']
# variable = 'perc_facil_general_p%s_c%s'
# figtest = GraficoCoeficientes1VarLR2(datadir, archivo, formulas, textformulas, variable,conf = 0.05)

In [33]:
def CorrerRegresiones1Var(df_reg, variable):
    rl = []
    coefs = []
    lb = []
    ub = []
    formulas = ['position ~ %s',
                'position ~ equipo + %s',
                'position~ Torneo + %s',
                'position ~ equipo + Torneo + %s']
    for formula in formulas:
        formula_ = formula % variable
#         mod = ols(formula = formula_, data = df_reg).fit()
        mod = ols(formula = formula_, data = df_reg).fit(cov_type = 'cluster',
                                                         cov_kwds={'groups': df_reg['equipo']})
        print(mod.summary())

In [34]:
datadir = os.path.join(os.pardir,
                       'Datos',
                       'Bases construidas',
                       'Bases main',
                       'Bases efecto')
archivo = 'Inglaterra.xlsx'
df_reg_t = pd.read_excel(os.path.join(datadir,archivo))
variable_t = 'perc_facil_general_p4_c3'
CorrerRegresiones1Var(df_reg_t, variable_t)

                            OLS Regression Results                            
Dep. Variable:               position   R-squared:                       0.004
Model:                            OLS   Adj. R-squared:                  0.002
Method:                 Least Squares   F-statistic:                     2.280
Date:                Mon, 15 Apr 2019   Prob (F-statistic):              0.138
Time:                        10:32:39   Log-Likelihood:                -1602.3
No. Observations:                 504   AIC:                             3209.
Df Residuals:                     502   BIC:                             3217.
Df Model:                           1                                         
Covariance Type:              cluster                                         
                               coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------------------
Intercept               

                            OLS Regression Results                            
Dep. Variable:               position   R-squared:                       0.006
Model:                            OLS   Adj. R-squared:                 -0.046
Method:                 Least Squares   F-statistic:                    0.3230
Date:                Mon, 15 Apr 2019   Prob (F-statistic):              0.998
Time:                        10:32:39   Log-Likelihood:                -1601.7
No. Observations:                 504   AIC:                             3255.
Df Residuals:                     478   BIC:                             3365.
Df Model:                          25                                         
Covariance Type:              cluster                                         
                                         coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------------------------
Inte

/home/cristian/anaconda3/lib/python3.7/site-packages/statsmodels/base/model.py:1532: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 49, but rank is 1
  'rank is %d' % (J, J_), ValueWarning)
/home/cristian/anaconda3/lib/python3.7/site-packages/statsmodels/base/model.py:1532: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 73, but rank is 25
  'rank is %d' % (J, J_), ValueWarning)


In [35]:
df_reg_t.columns

Index(['equipo', 'position', 'Torneo', 'perc_facil_general_p4_c3',
       'perc_dificil_general_p4_c3', 'perc_regular_general_p4_c3',
       'perc_facil_lv_p4_c3', 'perc_dificil_lv_p4_c3', 'perc_regular_lv_p4_c3',
       'facil_general_corr',
       ...
       'facil_general_corr.11', 'perc_facil_general_corr_p7_c5',
       'dificil_general_corr.11', 'perc_dificil_general_corr_p7_c5',
       'perc_regular_general_corr_p7_c5', 'facil_lv_corr.11',
       'perc_facil_lv_corr_p7_c5', 'dificil_lv_corr.11',
       'perc_dificil_lv_corr_p7_c5', 'perc_regular_lv_corr_p7_c5'],
      dtype='object', length=224)